# Exploratory Data Analysis of USBP Encounters

In [ ]:
%matplotlib inline

# load environment variables
from dotenv import load_dotenv
import os

load_dotenv()

## Visualize USBP monthly encounters data

In [ ]:
# load usbp encounters monthly data
import pandas as pd
import matplotlib
import matplotlib.style
import matplotlib.pyplot as plt

usbp_data_fp = os.getenv('USBP_DATA_FP')

monthly_counts_df = pd.read_excel(usbp_data_fp, sheet_name='Monthly Count')

# rename Fiscal Year column
monthly_counts_df = monthly_counts_df.rename(columns={'Fiscal\nYear': 'Fiscal Year'})
print(monthly_counts_df.head(n=12))

# 2025 data is incomplete - drop it for now
monthly_counts_df = monthly_counts_df.loc[(monthly_counts_df['Fiscal Year'] != 2025)]


In [ ]:
# plot unstacked bar charts of monthly usbp encounters for select years

years = [2000, 2010, 2020, 2024]

dfs = []
for year in years:
    year_df = monthly_counts_df.loc[(monthly_counts_df['Fiscal Year'] == year)].set_index('Month')
    year_df = year_df.rename(columns={'Quantity': f'{year}'}) # renamed for ease of plotting
    dfs.append(year_df)
    print(year_df.head(n=3))
years_df = pd.concat(dfs, axis=1) # concat dataframes horizontally

with plt.style.context('fivethirtyeight'): # see https://stackoverflow.com/questions/45103486/use-different-style-for-each-figure
    years_df.plot(kind='bar', y=[col for col in years_df.columns if col != 'Fiscal Year'], ylabel='Encounters', title='Monthly USBP Encounters')


In [ ]:
# plot line charts of total encounters for each month across all years

encounters_by_month = monthly_counts_df.groupby(['Month','Fiscal Year'])['Quantity'].sum()

with plt.style.context('bmh'):
    fig, axes = plt.subplots(nrows=4, ncols=3, figsize=(36, 36))
    plt.subplots_adjust(hspace=0.3)

    months = monthly_counts_df['Month'].unique()
    for i, month in enumerate(months):
        month_data = encounters_by_month[month]
        month_data.plot(kind='line', ax=axes[i // 3, i % 3], y='Quantity', ylabel='Encounters', title=f'USBP Encounters by Year for {month}')


In [ ]:
# plot unstacked bar chart of average usbp encounters for each month across all years

monthly_avgs = monthly_counts_df.groupby(by=['Month'])['Quantity'].mean()
print(monthly_avgs)

with plt.style.context('bmh'):
    monthly_avgs.plot(kind='bar', ylabel='Average Encounters', title='Average Monthly USBP Encounters')


In [ ]:
# plot unstacked bar chart of annual usbp encounters across all years

annual_totals = monthly_counts_df.groupby(by='Fiscal Year')['Quantity'].sum()
print(annual_totals)

with plt.style.context('Solarize_Light2'):
    annual_totals.plot(kind='bar', ylabel='Total Encounters', title='Annual USBP Encounters')
    plt.yscale('log')


## Visualize USBP regional monthly encounters data

In [ ]:
# load usbp encounters regional monthly data
import pandas as pd
import matplotlib
import matplotlib.style
import matplotlib.pyplot as plt

usbp_data_fp = os.getenv('USBP_DATA_FP')

reg_monthly_counts_df = pd.read_excel(usbp_data_fp, sheet_name='Monthly Region')

# rename Fiscal Year column
reg_monthly_counts_df = reg_monthly_counts_df.rename(columns={'Fiscal\nYear': 'Fiscal Year'})
print(reg_monthly_counts_df.head(n=12))

# 2025 data is incomplete - drop it for now
reg_monthly_counts_df = reg_monthly_counts_df.loc[(reg_monthly_counts_df['Fiscal Year'] != 2025)]


In [ ]:
# plot unstacked bar charts of usbp encounters across all months per region for select years

years = [2000, 2010, 2020, 2024]

with plt.style.context('ggplot'):
    fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(24, 24))
    plt.subplots_adjust(hspace=0.4)
    plt.yscale('log')

    for i, year in enumerate(years):
        year_data = reg_monthly_counts_df.loc[(reg_monthly_counts_df['Fiscal Year'] == year)].groupby(by='Region')['Quantity'].sum()
        print(f'Year: {year}\n{year_data}\n{'-'*45}')
        year_data.plot(kind='bar', ax=axes[i // 2, i % 2], y='Quantity', ylabel='Encounters', title=f'USBP Encounters by Region for {year}')


In [ ]:
# plot line chart of annual usbp encounters for each region

reg_annual_counts = reg_monthly_counts_df.groupby(by=['Region','Fiscal Year'])['Quantity'].sum()

with plt.style.context('seaborn-v0_8-pastel'):
    fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(36, 12))
    plt.subplots_adjust(wspace=0.4)
    plt.yscale('log')

    regions = reg_monthly_counts_df['Region'].unique()
    for i, region in enumerate(regions):
        annual_counts = reg_annual_counts[region]
        print(f'{region} annual encounters:\n{annual_counts.head(n=5)}\n{'-'*50}')
        annual_counts.plot(kind='line', ax=axes[i], y='Quantity', ylabel='Encounters', title=f'Annual USBP Encounters for {region}')


In [ ]:
# plot unstacked bar charts of monthly usbp encounters for each region across all years

data = reg_monthly_counts_df.groupby(by=['Region','Month'])['Quantity'].sum()

with plt.style.context('ggplot'):
    fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(36,36))
    plt.subplots_adjust(wspace=0.2)
    plt.yscale('log')
    
    regions = reg_monthly_counts_df['Region'].unique()
    for i, region in enumerate(regions):
        region_data = data[region]
        print(f'{region} total encounters per month across all years:\n{region_data.head(n=5)}\n{'-'*50}')
        region_data.plot(kind='bar', ax=axes[i], y='Quantity', ylabel='Encounters', title=f'Total Monthly USBP Encounters for {region}')
